In [ ]:
#!/bin/bash
# Kev @ 2019
# Bash script to calculate RMSD profile

[ $# -eq 0 ] && { echo "mkvmd> Usage: $0 [PDB] [TRJ] [PREFIX] [REF]"; exit 1; }

PDB="$1"
TRJ="$2"
PREFIX="$3"
REF="$4"

# SELREF="noh segname PROA"
# REGIONS=("noh segname PROA" "noh segname PROB" "noh segname PROC" "noh segname PROA and resid 1 to 180")
SELREF="noh chain A"
REGIONS=("noh chain A" "noh chain B" "noh chain C" "noh chain A and resid 1 to 180")
FILENAMES=("proa" "prob" "proc" "proahelix")


[ ! -z "PREFIX" ] && echo "mkvmd> Please specify prefix for output files"

if [ ! -n "$REF" ]; then
    REF="$PDB"
fi

files=("$PDB" "$TRJ" "$REF")
for file in "${files[@]}"; do
    if [ ! -f "$file" ]; then
        echo -e "$file \nStructure not found!"
        exit 1
    fi
done

cat > tcl <<EOF
mol new $PDB waitfor all
mol addfile $TRJ waitfor all
set num_frames [molinfo top get numframes]
set sel_all [atomselect top all]
mol new $REF waitfor all
set sel_ref0 [atomselect 1 "${SELREF}" frame 0]
set sel_ref [atomselect 0 "${SELREF}"]
EOF

counter=0
for region in "${REGIONS[@]}"; do
    filename="${PREFIX}-${FILENAMES[counter]}"
    rm $filename
    cat >> tcl <<EOF
set sel_rmsd0 [atomselect 1 "${region}" frame 0]
set sel_rmsd [atomselect 0 "${region}"]
set outfile [open $filename "w"]
puts \$outfile "# reference: $SELREF; calculated region: $region"
for {set i 0} {\$i<\$num_frames} {incr i} {
    \$sel_all frame \$i
    \$sel_ref frame \$i
    \$sel_rmsd frame \$i
    \$sel_all move [measure fit \$sel_ref \$sel_ref0]
    set rmsd [measure rmsd \$sel_rmsd \$sel_rmsd0 weight mass]
    puts \$outfile "\$i \t \$rmsd"
}
close \$outfile
EOF
((counter++))
done

cat >> tcl <<EOF
quit
EOF
vmd -dispdev text -e tcl

rm tcl

这个 Bash 脚本用于计算分子动力学（MD）轨迹文件中不同区域的 RMSD（均方根偏差）曲线。以下是逐步的中文解释：

### 1. 检查输入参数

```bash
[ $# -eq 0 ] && { echo "mkvmd> Usage: $0 [PDB] [TRJ] [PREFIX] [REF]"; exit 1; }
```

- 如果没有输入参数，脚本会提示需要的参数格式并退出。它需要四个参数：PDB 文件、轨迹文件、输出文件前缀（PREFIX）和参考结构（REF）。

### 2. 定义变量

```bash
PDB="$1"
TRJ="$2"
PREFIX="$3"
REF="$4"
```

- 将输入的第 1、2、3、4 个参数分别赋值给 `PDB`、`TRJ`、`PREFIX` 和 `REF` 变量。

### 3. 定义选择区域和文件名列表

```bash
SELREF="noh chain A"
REGIONS=("noh chain A" "noh chain B" "noh chain C" "noh chain A and resid 1 to 180")
FILENAMES=("proa" "prob" "proc" "proahelix")
```

- `SELREF` 定义参考结构的原子选择范围（如链 A 的非氢原子）。
- `REGIONS` 定义了需要计算 RMSD 的区域列表。
- `FILENAMES` 定义了每个区域对应的输出文件前缀。

### 4. 检查输出文件前缀是否为空

```bash
[ ! -z "PREFIX" ] && echo "mkvmd> Please specify prefix for output files"
```

- 如果 `PREFIX` 为空，脚本会提示需要提供输出文件前缀。

### 5. 设置默认参考结构文件

```bash
if [ ! -n "$REF" ]; then
    REF="$PDB"
fi
```

- 如果没有提供 `REF`，则使用 `PDB` 文件作为默认参考结构。

### 6. 检查文件是否存在

```bash
files=("$PDB" "$TRJ" "$REF")
for file in "${files[@]}"; do
    if [ ! -f "$file" ]; then
        echo -e "$file \nStructure not found!"
        exit 1
    fi
done
```

- 检查 `PDB`、`TRJ` 和 `REF` 文件是否存在。如果其中任何一个文件不存在，脚本会输出错误并退出。

### 7. 创建 VMD 脚本并写入 TCL 命令

```bash
cat > tcl <<EOF
mol new $PDB waitfor all
mol addfile $TRJ waitfor all
set num_frames [molinfo top get numframes]
set sel_all [atomselect top all]
mol new $REF waitfor all
set sel_ref0 [atomselect 1 "${SELREF}" frame 0]
set sel_ref [atomselect 0 "${SELREF}"]
EOF
```

- 将 TCL 命令写入名为 `tcl` 的文件。
- `mol new` 命令加载 PDB 文件和轨迹文件。
- `num_frames` 获取轨迹的总帧数。
- 设置 `sel_ref0` 为参考结构中的选择区域，`sel_ref` 为轨迹中的选择区域。

### 8. 循环计算各区域的 RMSD 并写入文件

```bash
counter=0
for region in "${REGIONS[@]}"; do
    filename="${PREFIX}-${FILENAMES[counter]}"
    rm $filename
    cat >> tcl <<EOF
set sel_rmsd0 [atomselect 1 "${region}" frame 0]
set sel_rmsd [atomselect 0 "${region}"]
set outfile [open $filename "w"]
puts \$outfile "# reference: $SELREF; calculated region: $region"
for {set i 0} {\$i<\$num_frames} {incr i} {
    \$sel_all frame \$i
    \$sel_ref frame \$i
    \$sel_rmsd frame \$i
    \$sel_all move [measure fit \$sel_ref \$sel_ref0]
    set rmsd [measure rmsd \$sel_rmsd \$sel_rmsd0 weight mass]
    puts \$outfile "\$i \t \$rmsd"
}
close \$outfile
EOF
((counter++))
done
```

- 脚本遍历 `REGIONS` 中的每个区域：
  - 为每个区域生成对应的文件名 `filename`。
  - 删除旧文件以确保重新写入数据。
  - 为每个区域选择区域 (`sel_rmsd0` 和 `sel_rmsd`)。
  - 计算 RMSD 值，并将帧编号和 RMSD 值写入文件。

### 9. 退出 VMD

```bash
cat >> tcl <<EOF
quit
EOF
```

- 在 TCL 脚本末尾添加 `quit` 命令，使 VMD 在执行完成后退出。

### 10. 执行 VMD 脚本

```bash
vmd -dispdev text -e tcl
```

- 使用 VMD 执行生成的 `tcl` 脚本，计算 RMSD 值。

### 11. 清理临时文件

```bash
rm tcl
```

- 删除临时的 `tcl` 文件以清理环境。

**------------------------------------------------------------------------------------------------------------------------------------------**


```bash
[ $# -eq 0 ] && { echo "mkvmd> Usage: $0 [PDB] [TRJ] [PREFIX] [REF]"; exit 1; }
```

### 逐个字符解释

1. **`[`** 和 **`]`**：
   - 用于表示条件测试的开始和结束。在 Bash 中，`[` 是 `test` 命令的同义词。

2. **`$#`**：
   - 表示传递给脚本的参数数量。`$#` 是一个特殊变量，返回命令行中提供的参数个数。

3. **`-eq`**：
   - 是一个比较运算符，表示“等于”。它用于比较两个数值。

4. **`0`**：
   - 是要与 `$#` 比较的数值。在这里，它表示没有提供任何参数。

5. **`&&`**：
   - 是逻辑与运算符，表示如果前面的条件为真（即 `$#` 等于 `0`），那么执行后面的命令。

6. **`{ ... }`**：
   - 大括号用于将多条命令组合在一起执行。

7. **`echo`**：
   - 是一个命令，用于在终端输出文本。这里它输出使用说明。

8. **`"mkvmd> Usage: $0 [PDB] [TRJ] [PREFIX] [REF]"`**：
   - 是 `echo` 命令的参数，输出的字符串。`$0` 是脚本的名称，它会被替换为实际的脚本名。

9. **`exit 1`**：
   - 是用于退出脚本的命令。`1` 是退出状态码，通常用来表示错误（`0` 表示成功）。

### 整体逻辑

- 整体逻辑是：如果没有传递任何参数（即 `$# -eq 0` 为真），那么就打印使用说明，并退出脚本，返回状态码 `1` 表示出错。